In [9]:
import pandas as pd
from matplotlib import pyplot as plt
#from dataenforce import Dataset
#DMarket_insight = Dataset["id", "date", ...]
data = pd.read_csv("../datasets/market_insights_overview_all_2021_11_12.csv")
data["date"] = pd.to_datetime(data["date"])


In [10]:
from utils import save_fig_for_raport
print(plt.style.available)
plt.style.use("seaborn")

NameError: name 'plt' is not defined

In [ ]:

from utils import dataframe_to_latex_tabular

In [ ]:
data.head()
data.info()

In [ ]:
categories = pd.read_csv("../datasets/solr_categories_all_16_09_2021.csv")
categories.head()

In [ ]:
# Join market_insight table with category names
category_names = categories[[ "internal_doc_id", "title"]]
category_id_to_names_map = dict(map(lambda x: (x["internal_doc_id"], x["title"]), category_names.to_dict("records")))
category_name_to_id_map = dict(map(lambda x: (x["title"], x["internal_doc_id"]), category_names.to_dict("records")))

In [ ]:
#print(category_names["title"])
#categories_file = open("categories.txt", "w")
#for cat_line in category_names["title"]:
#    categories_file.write(cat_line)
#    categories_file.write("\n")
#categories_file.close()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Set pandas to use decimal insteaf of scientific e^x notaion
pd.set_option("display.float_format", lambda x: "%.0f" % x)

interesting_data = data[["date", "hits", "clicks", "product_id", "cat_id"]]

print(f"Unique products: {data['product_id'].nunique()}")
print(f"Unique categories: {data['cat_id'].nunique()}")


grouped = interesting_data.groupby(["cat_id", "date"], as_index=False).sum()
# Filter away early 2018 data as it seems to contain errors
grouped = grouped.loc[grouped['date'] > '2018-12-01']

print(grouped)
grouped.describe()

In [ ]:
# Join with category names
grouped_joined= pd.merge(grouped, category_names, how="left", left_on=["cat_id"], right_on=["internal_doc_id"])
grouped_joined.rename(columns={"title": "cat_name"}, inplace=True)
grouped_joined.drop(columns=["internal_doc_id"], inplace=True, axis=1)

In [ ]:
dataframe_to_latex_tabular(
  grouped_joined.head().append({"cat_id": "...", "hits":"...", "date": "", "clicks":"...", "product_id": "...", "cat_name":"..."}, ignore_index=True).append(
    grouped_joined.tail()
  ),
  caption="Market Insights Overview dataset", 
  label="market_insights_overview_11-12-21")

NameError: name 'grouped_joined' is not defined

I perioden august 2018 til perioden august 2021 ble det registrert 1320 unike kategorier.
Et nevralnett med 1320 i bredde er litt stort kanskje? Det vil også låse modellen til X antall kategorier.
Dette er et argument for å se på hver kategori inviduelt og ha en egen modell per kategori.

Ulempen med det igjen er at det blir veldig mange modeller...

In [ ]:
print(grouped[grouped["hits"] == 1].count())
grouped.boxplot(column=["hits", "clicks"])
grouped.hist(column=["hits", "clicks"], range=(0, 50))

The boxplot shows that on most days hits hower around 10 500, and clicks around 8000. There have been one extreme case on 1 day, where 
1 category had 60 000 hits!
The histogram show the distribution, a very few days a year a few categories has a lot of hits.
The longer to the right we get the more days and categories are represented.

In [ ]:
import matplotlib.dates as mdate
date_locator = mdate.MonthLocator()
formatter = mdate.DateFormatter("%Y-%m")
all_categories = data["cat_id"].unique()
for cat in all_categories[:15]:
    cat_name = categories.loc[categories["internal_doc_id"] == cat]["title"]
    save_fig_for_raport(
    grouped[grouped["cat_id"] == cat].plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori_id: {cat_name}", 
        figsize=(30, 10),
        ),f"lineplot_{cat}_{category_id_to_names_map[cat]}")
#interesting_data.plot()
        #.xaxis.set_major_locator(date_locator)
#grouped.plot(x="date", y=["hits", "clicks"],)

In [ ]:
turmat = grouped[grouped["cat_id"] == 11488]
turmat.head()
name = categories.loc[categories["internal_doc_id"] == 11488]
name.head()
save_fig_for_raport(
        turmat.plot(
                x="date", 
                y=["hits", "clicks"], 
                title=f"categori: Turmat", 
                figsize=(30, 10),
        ),f"lineplot_11488_turmat")

In [ ]:
vapenskap = grouped[grouped["cat_id"] == 12738]
vapenskap.head()
name = categories.loc[categories["internal_doc_id"] == 12738]
name.head()
save_fig_for_raport(
        vapenskap.plot(
                x="date", 
                y=["hits", "clicks"], 
                title=f"categori: Våpenskap", 
                figsize=(30, 10)),
        f"lineplot_12738_vapenskap"
        )


In [ ]:
kopp_og_krus = grouped[grouped["cat_id"] == 12663]
kopp_og_krus.head()
name = categories.loc[categories["internal_doc_id"] == 12663]
name.head()
kopp_og_krus.plot(
        x="date", 
        y=["hits", "clicks"], 
        title=f"categori: Kopp og krus", 
        figsize=(30, 10),)


Det virker ikke som om det er data på kopper og krus. Dette burde undersøkes nærmere. Forhåpentligvis finnes dataen, men den er ikke hentet ut og sendt til oss.

In [ ]:
hengekøye = grouped[grouped["cat_id"] == 11514]
hengekøye.head()
name = categories.loc[categories["internal_doc_id"] == 11514]
name.head()
save_fig_for_raport(

        hengekøye.plot(
                x="date", 
                y=["hits", "clicks"], 
                title=f"categori: Hengekøye", 
                figsize=(30, 10),),
        f"lineplot_11514 hengekøye"
)

In [ ]:
gpu = grouped[grouped["cat_id"] == 30]
gpu.head()
name = categories.loc[categories["internal_doc_id"] == 30]
name.head()
save_fig_for_raport(
        gpu.plot(
                x="date", 
                y=["hits", "clicks"], 
                title=f"categori: {category_id_to_names_map[30]}", 
                figsize=(30, 10),),
        f"lineplot_30_gpu"
)

In [ ]:
import numpy as np
name.head()
import seaborn as sns
print(np.corrcoef(data["hits"], data["clicks"]))
correlation_matrix = data[["hits", "clicks"]].corr()
sns.heatmap(correlation_matrix)

Høy korrelasjon mellom hits og klicks!

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.hist(figsize=(20,15))

In [ ]:

# correlation_matrix = data.corr()
# sns.heatmap(correlation_matrix)
data_by_cat_id = grouped.pivot(index=["date"], columns=["cat_id"])["hits"]
data_by_cat_id.tail()

In [ ]:
# Check the validity of the pivot table
grouped_joined[(grouped_joined['date'] == '2021-09-12') & (grouped_joined['cat_id'] == 13759)]
# Seems to check out

In [ ]:
import seaborn as sns
# Require at least 200 observations for correlation to be valid
category_correlation = data_by_cat_id.corr(min_periods=100)
plt.figure(figsize=(20,10))
save_fig_for_raport(
  sns.heatmap(category_correlation),
  "category_correlation_matrix"  
)
category_correlation

In [ ]:
category_correlation.to_csv("../datasets/category_correlation.csv")

In [ ]:
# Find number of correlations with bærbar-pc som ikke er 0 eller 1
pc_correlations = category_correlation[2].dropna()
print("Number of correlations with bærbar pc which is not 0 or 1: ",
  len(
  list(
    filter(lambda x: x != 0 and x != 1, 
       pc_correlations
      )
  )
))
print("Number of correlations with bærbar pc which is 1: ",list(filter(lambda x: x == 1, pc_correlations)))

In [ ]:
cpu_id = category_name_to_id_map['Prosessor (CPU)']
gpu_id = category_name_to_id_map['Grafikkort (GPU)']
cpu_gpu_correlation = data_by_cat_id.loc[:, [cpu_id, gpu_id]].corr()
print(cpu_gpu_correlation)
data_by_cat_id.loc[:, [cpu_id, gpu_id]]

Ser ikke ut som CPU eller GPU korrelerer i det hele tatt

In [ ]:
print(category_id_to_names_map[6])
print(category_id_to_names_map[2])
two_six = data_by_cat_id.loc[:, [2, 6]]
two_six.plot()
two_six.plot.scatter(x=6,y=2)
two_six_corr =two_six.corr()
print(two_six_corr)
two_six

Hvorfor korrelerer disse 100%?

In [ ]:

print(category_id_to_names_map[13759])
print(category_id_to_names_map[2])
dopapir_pc = data_by_cat_id.loc[:, [2,  13759]]
dopapir_pc.plot()
dopapir_pc_corr = dopapir_pc.corr()
print(dopapir_pc_corr)
dopapir_pc

Ser ut som dopapir og PC kun korrelerer 100 % fordi dopapir er en ny kategori

In [ ]:
# Save some statistics for Mobiltelefon
dataframe_to_latex_tabular(
  grouped_joined[grouped_joined["cat_id"] == category_name_to_id_map['Mobiltelefon']].drop("product_id", axis=1).describe(),
  caption="Mobiltelefon statistics",
  label="Mobiltelefon_statistics",
  add_index=True
)

## How many categories has 0 values in a given day?

In [ ]:
save_fig_for_raport(
  data_by_cat_id.isna().sum().plot(),
  name="category_NaN_values"
)
data_by_cat_id.isna().sum()

In [ ]:
save_fig_for_raport(
    data_by_cat_id.isin([0]).sum().plot(),
    name="category_0_values"
 )
data_by_cat_id.isin([0]).sum()

## Lag plot

resource: http://www.itl.nist.gov/div898/handbook/eda/section3/lagplot.htm

In the graph below, firt axis represents the t(lag), seond axis represents t+1

Ex: if data is, [1,4,5,3,2], then y(t):= [1,4,5,3,2], y(t+1): = [4,5,3,2]

As we see the graph below, this suggests the non-random pattern (graph is poistively linear).

Non-randomness in the data reveals that we could use an autoregressive model

In [ ]:
from pandas.plotting import lag_plot

In [ ]:
lag_plot(grouped_joined[grouped_joined["cat_id"] == category_name_to_id_map['Mobiltelefon']]["clicks"])
for cat_id in all_categories[:10]:
  save_fig_for_raport(
    lag_plot(grouped_joined[grouped_joined["cat_id"] == cat_id]["hits"]),
    name=f"lag_plot_hits_10_categories"
  )

In [ ]:

lag_plot(
  grouped_joined[grouped_joined["cat_id"] == category_name_to_id_map['Mobiltelefon']]["clicks"],
  lag=7
  )

https://www.itl.nist.gov/div898/handbook/eda/section3/lagplot.htm

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(grouped_joined[grouped_joined["cat_id"] == category_name_to_id_map['Mobiltelefon']]["hits"])
autocorrelation_plot(grouped_joined[grouped_joined["cat_id"] == category_name_to_id_map['Mobiltelefon']]["hits"], 
  label="hits",
  )

resource: http://www.itl.nist.gov/div898/handbook/eda/section3/eda35c.htm

A lag plot above shows the structure in the data. In order to quantify the correlation between the point at t and point at t+1 withe respect to expectation, autocrrelation is used.

A black line in the graph below shows the expectation for random data(thus 0 correlation) and two dash lines above and below it represent the confidence interval with each 95% and 99%.

The graph shows a strong correlation for lags of < 100 days. (lag 0 is always 1 corrleation)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(
  grouped_joined[grouped_joined["cat_id"] == category_name_to_id_map['Mobiltelefon']]["hits"],
  lags=7
  )